In [21]:
import requests
import json
import pandas as pd
import os
import numpy as np

from google.cloud import bigquery
from google.oauth2 import service_account

#external modules
import api_call_helpers as ach
import bigquery_functions as pp
# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# bigquery credentials
path = os.getcwd()
secrets_filename = 'bigquery_auth.json'

CREDENTIALS = service_account.Credentials.from_service_account_file(
    f'{path}/{secrets_filename}')
PROJECT_ID = 'riotdashboard-283414'
DATASET_ID = 'RIOT_DWH'
TABLE_ID = 'MATCH_STATISTICS'

In [3]:
# set up riot params
USERNAME = 'tanas0rn'
API_KEY = 'RGAPI-f0473e1a-e5e8-4334-9252-7828b0d625c3'
ORIGIN = "https://developer.riotgames.com"
ACCEPT_CHARSET = "application/x-www-form-urlencoded; charset=UTF-8"
ACCEPT_LANGUAGE = "de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7,en-GB;q=0.6"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"

headers = {
    "User-Agent": USER_AGENT,
    "Accept-Language": ACCEPT_LANGUAGE,
    "Accept-Charset": ACCEPT_CHARSET,
    "Origin": ORIGIN,
    "X-Riot-Token": API_KEY
}

# Connect to bigquery 
(get existing matches so that we dont insert matches already in the table)

In [110]:
# connect to BigQuery 
# get list of all existing matches/players
client = bigquery.Client(credentials= CREDENTIALS,project=PROJECT_ID)

query = f"""
SELECT *
FROM RIOT_DWH.MATCH_STATISTICS
WHERE summonername = '{USERNAME}';
"""

query_job = client.query(query)
results = query_job.result().to_dataframe() 

# Query riot api

In [111]:
# get summoner info (account ID etc)
encrypted_account_id = ach.get_encrypted_accid(USERNAME,headers)

 Request Successful


In [112]:
# get match list for summoner
match_list = ach.get_match_list_by_accid(encrypted_account_id,headers)

 Request Successful
 Request Successful
 Request Successful
Total 255 Matches Retrieved


In [113]:
# get all of the match IDs
match_ids = [item['gameId'] for item in match_list]

# make sure that this record doesn't already exist in database (take out ones that already exist)
match_ids = np.setdiff1d(match_ids,list(results['matchID']))

In [114]:
len(match_ids)

218

In [135]:
# get match stats
matchstats = []
for match_id in match_ids[:50]:
    stats = ach.get_match_stats(USERNAME,match_id,headers)
    
    matchstats.append(stats)

 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful


# Data preprocessing (to tuples + remove unwanted fields + ensure data integrity)

df = pd.DataFrame(matchstats)
for col in df.columns:
    print(col)

In [238]:
db_columns = list(results.columns)

In [252]:
# null value handling
for match in matchstats:
    keys_to_remove = set(match.keys()) - set(db_columns)
    pp.entries_to_remove(keys_to_remove,match)
    
    for col in db_columns:
        if col in match.keys():
            pass
        else:
            match[col] = None
    assert len(match) == 59

In [277]:
#convert to tuples and assert column order
# we have to convert this in a billion formats for the datatype
df = pd.DataFrame(matchstats, columns = db_columns)
df = df.replace({np.nan: None})
records = df.to_dict('records')
matchstats_tuples = [tuple(d.values()) for d in records]

# Insert data into bigquery table

In [290]:
pp.insert_rows_to_bigquery(CREDENTIALS,
                            PROJECT_ID,
                            DATASET_ID,
                            TABLE_ID,
                            matchstats_tuples)

[]
